In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [33]:
# Apriori 算法
# 1. 支持度计算
def loadDataSet():
        # 读取数据txt中的数据
        data = pd.read_csv('Transactions.csv')
        return data

In [18]:
def createC1(dataSet):# 创建候选项集
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))# 使用frozenset是为了后面可以将这些值作为字典的键

def scanD(D, Ck, minSupport):# 从候选项集中筛选出满足最小支持度的项集
    ssCnt = {}

    for tid in D: # 遍历数据集中的每一条交易记录
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: # 如果不在字典中，就添加进去
                    ssCnt[can] = 1
                else: # 如果在字典中，就加1
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData



In [19]:
# 2. 生成候选项集
def aprioriGen(Lk, k):# 生成候选项集
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

def apriori(dataSet, minSupport = 0.5): # 生成所有满足最小支持度的项集
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData


In [20]:
# 3. 生成关联规则
def generateRules(L, supportData, minConf=0.7):# 生成关联规则
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

def calcConf(freqSet, H, supportData, brl, minConf=0.7):# 计算置信度
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet-conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):# 生成候选规则集
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

    

In [35]:
# test
dataSet = loadDataSet()
dataSet.head()


,COD,pasta,milk,water,biscuits,coffee,brioches,yoghurt,frozen vegetables,tunny,...,tomato souce,coke,rice,juices,crackers,oil,frozen fish,ice cream,mozzarella,tinned meat
0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
1,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,4,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [36]:
C1 = createC1(dataSet)
D = list(map(set, dataSet))
L1, supportData0 = scanD(D, C1, 0.5)
print(L1)
print(supportData0)

[frozenset({'e'})]
{frozenset({'C'}): 0.047619047619047616, frozenset({'D'}): 0.047619047619047616, frozenset({'O'}): 0.047619047619047616, frozenset({' '}): 0.2857142857142857, frozenset({'a'}): 0.38095238095238093, frozenset({'p'}): 0.047619047619047616, frozenset({'s'}): 0.38095238095238093, frozenset({'t'}): 0.38095238095238093, frozenset({'i'}): 0.42857142857142855, frozenset({'k'}): 0.14285714285714285, frozenset({'l'}): 0.19047619047619047, frozenset({'m'}): 0.23809523809523808, frozenset({'e'}): 0.6666666666666666, frozenset({'r'}): 0.47619047619047616, frozenset({'w'}): 0.047619047619047616, frozenset({'b'}): 0.19047619047619047, frozenset({'c'}): 0.42857142857142855, frozenset({'u'}): 0.23809523809523808, frozenset({'f'}): 0.14285714285714285, frozenset({'o'}): 0.42857142857142855, frozenset({'h'}): 0.14285714285714285, frozenset({'g'}): 0.09523809523809523, frozenset({'y'}): 0.09523809523809523, frozenset({'n'}): 0.19047619047619047, frozenset({'v'}): 0.047619047619047616, f

In [37]:
L, supportData = apriori(dataSet)
print(L)
print(supportData)

[[frozenset({'e'})], []]
{frozenset({'C'}): 0.047619047619047616, frozenset({'D'}): 0.047619047619047616, frozenset({'O'}): 0.047619047619047616, frozenset({' '}): 0.2857142857142857, frozenset({'a'}): 0.38095238095238093, frozenset({'p'}): 0.047619047619047616, frozenset({'s'}): 0.38095238095238093, frozenset({'t'}): 0.38095238095238093, frozenset({'i'}): 0.42857142857142855, frozenset({'k'}): 0.14285714285714285, frozenset({'l'}): 0.19047619047619047, frozenset({'m'}): 0.23809523809523808, frozenset({'e'}): 0.6666666666666666, frozenset({'r'}): 0.47619047619047616, frozenset({'w'}): 0.047619047619047616, frozenset({'b'}): 0.19047619047619047, frozenset({'c'}): 0.42857142857142855, frozenset({'u'}): 0.23809523809523808, frozenset({'f'}): 0.14285714285714285, frozenset({'o'}): 0.42857142857142855, frozenset({'h'}): 0.14285714285714285, frozenset({'g'}): 0.09523809523809523, frozenset({'y'}): 0.09523809523809523, frozenset({'n'}): 0.19047619047619047, frozenset({'v'}): 0.047619047619047

In [38]:
rules = generateRules(L, supportData, minConf=0.7)
print(rules)

[]
